In [1]:
pip install pyspark


     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 46.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=6d957aa5055c4939433451c80784d84271489d2f4e7c5ab271f28cacaac017f7
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("projecyt").getOrCreate()

In [4]:

df = spark.read.csv("/content/drive/MyDrive/BigData/iris.data",header=True,inferSchema=True)
df.printSchema()

root
 |-- 5.1: double (nullable = true)
 |-- 3.5: double (nullable = true)
 |-- 1.4: double (nullable = true)
 |-- 0.2: double (nullable = true)
 |-- Iris-setosa: string (nullable = true)



In [5]:
k=len(df.columns)
for i in range(k):
  old_name = df.columns[i]
  new_name = "f"+str(i)
  print(new_name)
  df=df.withColumnRenamed(old_name,new_name)

f0
f1
f2
f3
f4


In [6]:
df.show()

+---+---+---+---+-----------+
| f0| f1| f2| f3|         f4|
+---+---+---+---+-----------+
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
|5.4|3.7|1.5|0.2|Iris-setosa|
|4.8|3.4|1.6|0.2|Iris-setosa|
|4.8|3.0|1.4|0.1|Iris-setosa|
|4.3|3.0|1.1|0.1|Iris-setosa|
|5.8|4.0|1.2|0.2|Iris-setosa|
|5.7|4.4|1.5|0.4|Iris-setosa|
|5.4|3.9|1.3|0.4|Iris-setosa|
|5.1|3.5|1.4|0.3|Iris-setosa|
|5.7|3.8|1.7|0.3|Iris-setosa|
|5.1|3.8|1.5|0.3|Iris-setosa|
|5.4|3.4|1.7|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 20 rows



In [7]:
import pandas as pd
pd.DataFrame(df.take(5),columns=df.columns).transpose()

,0,1,2,3,4
f0,4.9,4.7,4.6,5,5.4
f1,3,3.2,3.1,3.6,3.9
f2,1.4,1.3,1.5,1.4,1.7
f3,0.2,0.2,0.2,0.2,0.4
f4,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa


In [8]:
class_name = df.columns[len(df.columns)-1]
df.groupBy(class_name).count().toPandas()
print(class_name)

f4


In [9]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol("label")
df1=indexer.fit(df).transform(df)

In [10]:

df1.show()


+---+---+---+---+-----------+-----+
| f0| f1| f2| f3|         f4|label|
+---+---+---+---+-----------+-----+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|
|5.4|3.9|1.3|0.4|Iris-setosa|  2.0|
|5.1|3.5|1.4|0.3|Iris-setosa|  2.0|
|5.7|3.8|1.7|0.3|Iris-setosa|  2.0|
|5.1|3.8|1.5|0.3|Iris-setosa|  2.0|
|5.4|3.4|1.7|0.2|Iris-setosa|  2.0|
+---+---+---+---+-----------+-----+
only showing top 20 rows



In [11]:

(training_data,test_data)=df1.randomSplit([0.8,0.2])

In [12]:
training_data.toPandas()

,f0,f1,f2,f3,f4,label
0,4.3,3.0,1.1,0.1,Iris-setosa,2.0
1,4.4,2.9,1.4,0.2,Iris-setosa,2.0
2,4.4,3.0,1.3,0.2,Iris-setosa,2.0
3,4.4,3.2,1.3,0.2,Iris-setosa,2.0
4,4.6,3.1,1.5,0.2,Iris-setosa,2.0
...,...,...,...,...,...,...
124,7.7,2.6,6.9,2.3,Iris-virginica,1.0
125,7.7,2.8,6.7,2.0,Iris-virginica,1.0
126,7.7,3.0,6.1,2.3,Iris-virginica,1.0
127,7.7,3.8,6.7,2.2,Iris-virginica,1.0


In [13]:
class_name="label"
feature_name=df.columns[:-1]
print(class_name)
print(feature_name)

label
['f0', 'f1', 'f2', 'f3']


In [15]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_name).setOutputCol("features")
transformed_data = assembler.transform(df1)

In [16]:
transformed_data.toPandas()

,f0,f1,f2,f3,f4,label,features
0,4.9,3.0,1.4,0.2,Iris-setosa,2.0,"[4.9, 3.0, 1.4, 0.2]"
1,4.7,3.2,1.3,0.2,Iris-setosa,2.0,"[4.7, 3.2, 1.3, 0.2]"
2,4.6,3.1,1.5,0.2,Iris-setosa,2.0,"[4.6, 3.1, 1.5, 0.2]"
3,5.0,3.6,1.4,0.2,Iris-setosa,2.0,"[5.0, 3.6, 1.4, 0.2]"
4,5.4,3.9,1.7,0.4,Iris-setosa,2.0,"[5.4, 3.9, 1.7, 0.4]"
...,...,...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica,1.0,"[6.7, 3.0, 5.2, 2.3]"
145,6.3,2.5,5.0,1.9,Iris-virginica,1.0,"[6.3, 2.5, 5.0, 1.9]"
146,6.5,3.0,5.2,2.0,Iris-virginica,1.0,"[6.5, 3.0, 5.2, 2.0]"
147,6.2,3.4,5.4,2.3,Iris-virginica,1.0,"[6.2, 3.4, 5.4, 2.3]"


In [17]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])


In [18]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [19]:

M = model.fit(training_data)

In [20]:
predictions = M.transform(test_data)

In [21]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.9736842105263158
